In [2]:
from functions import *

In [86]:
df = load_airtable_data()
df.to_csv('Data/airtable_data.csv', index=False)

In [3]:
df = pd.read_csv('Data/airtable_preprocessed_data.csv')

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

def success_rate(df):

    # Filter out rows where starting time is in the future
    df['starting_time'] = pd.to_datetime(df['starting_time'], errors='coerce')
    df_now = df[df['starting_time'] < datetime.now()].copy()

    # Filter out rows with missing or invalid ticket price information
    df_now['ticket_price'] = df_now['ticket_price'].fillna('')
    df_now['remain_prices'] = df_now['remain_prices'].fillna('')
    filtered_df = df_now[
        (df_now['ticket_price'] != 'No information available') &
        (df_now['ticket_price'] != '0.0')
    ].copy()

    # Process ticket prices and remaining prices into lists
    filtered_df['ticket_price_list'] = filtered_df['ticket_price'].str.split(', ').apply(
        lambda x: [float(price) for price in x if price.replace('.', '', 1).isdigit()]
    )
    filtered_df['remain_prices_list'] = filtered_df['remain_prices'].str.split(', ').apply(
        lambda x: [float(price) if price != 'SOLD OUT' else 0.0 for price in x]
    )

    # Calculate mean ticket prices and remaining prices
    filtered_df['mean_ticket_price'] = filtered_df['ticket_price_list'].apply(lambda x: round(np.mean(x), 2))
    filtered_df['mean_remain_price'] = filtered_df.apply(
    lambda row: round(sum(row['remain_prices_list']) / len(row['ticket_price_list']), 2),
    axis=1
)

    # Calculate success rate
    filtered_df['success_rate'] = filtered_df.apply(
        lambda row: abs(round(1 - (row['mean_remain_price'] / row['mean_ticket_price']), 2)), axis=1
    )

    # Return the success rates as a list
    return filtered_df


In [6]:
success_rate(df).shape

(356, 29)

In [217]:
df_success = success_rate(df)[['event_title', 'ticket_price','mean_ticket_price', 'remain_prices', 'mean_remain_price' ,'success_rate']]

In [222]:
df_success = df_success[(df_success['success_rate'] > 0.01) & (df_success['success_rate'] < 0.99)]


In [223]:
df_success

,event_title,ticket_price,mean_ticket_price,remain_prices,mean_remain_price,success_rate
17,Daria Kolosova + Prauze + Hard techno (the cube),"10.0, 15.0, 20.0, 25.0, 30.0, 50.0",25.00,"10.0, 20.0, 25.0, 30.0, 50.0",22.50,0.10
40,Viernes 29/11 ROTO + ANTIDOTO CLUB + RAKS CLUB...,"10.0, 12.0, 15.0, 18.0, 20.0, 22.0, 28.0",17.86,"18.0, 20.0, 22.0, 28.0",12.57,0.30
41,JALEO VILLAGE VALENCIA.,"2.0, 5.0, 10.0, 50.0",16.75,"5.0, 10.0, 50.0",16.25,0.03
44,VIERNES | TORMENTO CLUB,"0.0, 10.0, 15.0, 19.0, 20.0",12.80,"0.0, 5.0, 10.0, 15.0, 19.0, 20.0",13.80,0.08
50,Shlomo + Impuls w/ Chontane + Wateke w/ Neverdogs,"15.0, 20.0, 25.0, 30.0",22.50,"25.0, 30.0",13.75,0.39
51,SAT 30 NOV | MAR ELECTRONIC XXL,"12.0, 15.0, 18.0, 20.0, 23.0, 39.0, 60.0, 700.0",110.88,"30.0, 35.0, 60.0, 500.0",78.12,0.30
52,Verbena 30.11.2024,"0.0, 10.0, 15.0, 120.0",36.25,"0.0, 10.0, 120.0",32.50,0.10
55,FAYER,"0.0, 15.0, 150.0, 250.0, 300.0, 350.0, 400.0, ...",285.00,"0.0, 15.0, 200.0, 250.0, 300.0, 350.0, 400.0, ...",290.56,0.02
56,Estació Sonora,"2.0, 16.0, 19.0",12.33,"2.0, 14.0, 15.0, 18.0",16.33,0.32
60,SÁBADO NOCHE | ISMAELO + CARLITTOS + CENCII,"0.0, 10.0, 15.0, 19.0, 20.0",12.80,"0.0, 5.0, 10.0, 15.0, 19.0, 20.0",13.80,0.08
